In [1]:
# importing relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', None, 'max_colwidth', 250)

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


First we must load our dataset, isolate our dependent and independent variables, and split the dataset into train, validation, and test sets.

In [4]:
# reading in csv and confirmation          
reddit = pd.read_csv('sample_comments.csv', index_col=[0])

reddit.head()

,label,comment
139484,0,Nope.
638093,0,Donde esta la biblioteca?
662570,0,I feel bad for the mummy... not even the minions want to get close to him :(
652534,1,"The Pirate Bay, of course!"
411846,1,That's exactly how multiple photographers ended up on the pavement; they tripped.


In [5]:
# setting X and y
X = reddit['comment']
y = reddit['label']

In [6]:
# first split into test and remainder
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [7]:
# second split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, test_size = 0.25, random_state=42)

Now that we have loaded our data and performed a `train_test_split`, we can tokenize and vectorize our comments. We will be using the `TFIDF` vectorizer as it is a a better match with the `KNN` model than the `bagofwords`.

Below is a custom tokenizer function which we will pass through our TFIDF vectorizer. It splits the comments into words, eliminates numbers and punctuation, and stems the ends of words, leaving just the root.

In [8]:
#instantiating stemmer and stop words
stemmer = nltk.stem.PorterStemmer()
stop_words = stopwords.words('english')

# custom tokenizer function
def tokenizer(sentence):
    
    num_range = range(0,100) # num range to loop through
    
    for num in num_range: # looping through num range to filter out numbers
        sentence = sentence.replace(str(num), '')
    
    # removing punctuation and setting to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()
        
    
    # splitting sentence into words
    words_list = sentence.split(' ')
    
    tokens = [] # list to be filled with tokens
    
    
    # removing stopwords and any empty strings
    for word in words_list:
        if (not word in stop_words) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            tokens.append(stemmed_word)

    return tokens

We want our tokens to be present in at least 25 documents, be split using our custom tokenizer, and to be left with only the 1500 most prevalent tokens.

In [9]:
# instantiaing tfidf with custom tokenizer
tfidf = TfidfVectorizer(min_df=25, tokenizer=tokenizer, max_features=1500, dtype=np.float32)

# fitting to data
tfidf.fit(X_train)

# transformations 
train_tokens = tfidf.transform(X_train)
val_tokens = tfidf.transform(X_val)
test_tokens = tfidf.transform(X_test)

C:\Users\gravi\anaconda3\envs\sarcasm\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Having both tokenized and vectorized our comments, we will turn them into a dataframe both to briefly examine them, and to pass them through our KNN model.

In [10]:
# turning tokens + vectors into dataframe
train_tfidf = pd.DataFrame(columns=tfidf.get_feature_names(), data=train_tokens.toarray())

train_tfidf.head()

,abil,abl,abort,absolut,abus,accept,access,accord,account,accur,across,act,action,activ,actual,ad,add,addict,admit,adult,advic,affect,afford,age,ago,agre,ah,ahead,aim,aint,air,album,alcohol,alien,aliv,allow,almost,alon,along,alreadi,alright,also,altern,alway,amaz,america,american,amirit,amount,amp,angri,anim,announc,annoy,anoth,answer,anymor,anyon,anyth,anyway,apart,app,appar,appear,appl,appli,appreci,appropri,area,arent,argu,argument,arm,armi,armor,around,arrest,art,articl,artist,asian,ask,ass,assault,asshol,assum,atheist,attack,attempt,attent,attract,australia,avail,averag,avoid,aw,awar,away,awesom,b,babi,back,bad,bag,balanc,ball,ban,bank,bar,base,basic,batteri,battl,bear,beat,beauti,becom,beer,begin,behind,believ,benefit,berni,besid,best,bet,beta,better,big,bigger,biggest,bike,bill,bit,bitch,black,blame,blind,block,blood,blow,blue,board,bodi,bomb,book,boot,bore,born,boss,bot,bother,bottl,bottom,bought,bowl,box,boy,brain,brand,break,bring,british,bro,broke,broken,brother,brought,brown,buddi,buff,bug,build,built,bullet,bullshit,bunch,burn,bush,busi,butt,button,buy,c,call,came,camera,camp,campaign,canada,canadian,cancer,candid,cannot,cant,cap,capit,car,card,care,career,carri,case,cash,casual,cat,catch,caught,caus,center,certain,certainli,challeng,champion,chanc,chang,channel,charact,charg,cheap,cheat,check,chees,chicken,child,children,china,chines,choic,choos,christian,christma,church,citi,citizen,civil,claim,class,classic,clean,clear,clearli,click,clinton,close,cloth,club,coach,code,cold,collect,colleg,color,come,comic,comment,commi,commit,common,commun,communist,compani,compar,competit,complain,complet,comput,concern,confirm,confus,connect,conserv,consid,consol,conspiraci,construct,content,continu,contract,contribut,control,convers,convinc,cool,cop,copi,corner,corpor,correct,corrupt,cost,could,couldnt,count,countri,coupl,cours,court,cover,crap,crash,crazi,creat,credit,creepi,cri,crime,crimin,critic,cross,cultur,cup,current,custom,cut,cute,cuz,da,dad,daili,damag,damn,danger,dare,dark,dat,data,date,day,de,dead,deal,death,debat,decent,decid,decis,deck,deep,defend,defens,definit,degre,delet,demand,democraci,democrat,depend,depress,deserv,design,destroy,develop,dick,didnt,die,differ,difficult,direct,dirti,disagre,discuss,disgust,dlc,doctor,doesnt,dog,dollar,donald,donat,done,dont,door,doubl,doubt,download,downvot,draft,dragon,draw,dream,dress,drink,drive,driver,drop,drug,drunk,dude,due,duh,dumb,dunno,e,earli,earth,easi,easier,easili,east,eat,econom,economi,edit,educ,effect,effort,egg,eh,either,elect,els,em,email,emot,employe,end,enemi,energi,engin,english,enjoy,enough,entir,episod,equal,especi,etc,eu,europ,european,even,event,ever,everi,everybodi,everyon,everyth,everywher,evid,evil,exact,exactli,exampl,excel,except,excit,exclus,excus,exist,expect,expens,experi,expert,explain,extra,extrem,eye,f,face,facebook,fact,fail,fair,faith,fake,fall,fals,famili,fan,fantast,far,farm,fast,faster,fat,father,fault,favorit,fear,featur,feed,feel,felt,femal,feminist,field,fight,figur,file,fill,film,filthi,final,find,fine,finger,finish,fire,first,fit,five,fix,flag,flair,fli,flip,folk,follow,food,footbal,forc,foreign,forev,forget,forgot,form,forward,found,four,fox,fp,frame,free,freedom,french,friend,front,fuck,fuckin,full,fun,fund,funni,futur,g,ga,gain,game,garbag,gave,gay,gb,gear,gender,gener,german,germani,get,giant,gif,girl,girlfriend,give,given,glad,glass,global,go,goal,god,goe,gold,gone,gonna,good,googl,got,gotta,gotten,govern,grab,graphic,great,greatest,green,ground,group,grow,guess,guilti,gun,guy,ha,hack,haha,hair,half,hand,handl,hang,happen,happi,hard,harder,hasnt,hat,hate,havent,he,head,health,healthi,hear,heard,heart,heat,hed,hell,help,hero,hey,hide,high,higher,hilari,hillari,hire,histori,hit,hitler,hmm,hold,hole,holi,home,honest,honestli,hope,horribl,hot,hour,hous,huge,huh,human,hurt,hype,ice,id,idea,idiot,idk,ignor,ill,illeg,im,imag,imagin,immedi,immigr,imo,impli,import,imposs,impress,improv,includ,increas,incred,inde,individu,industri,in

In [11]:
# verifying shape
train_tfidf.shape

(142131, 1500)

In [12]:
# validation into dataframe
val_tfidf = pd.DataFrame(columns=tfidf.get_feature_names(), data=val_tokens.toarray())

val_tfidf.shape

(47378, 1500)

In [13]:
# test into dataframe
test_tfidf = pd.DataFrame(columns=tfidf.get_feature_names(), data=test_tokens.toarray())

test_tfidf.shape

(63170, 1500)

We are ready to run our data through the KNN model. We will be using the metric of `cosine`, which represents `cosine similarity`. Cosine similarity measures the relationship between data points through their relative angle. An angle closer to zero indicates a greater similarity, while an angle of 90 indicates no relationship. This is an alternative to using a `distance` measure, which would calculate how far apart data points are from one another. Cosine similarity is the best measure in our case, and is generally better suited for text data.

In [14]:
# instantiating knn model and fitting to train
KNN_model = KNeighborsClassifier(metric='cosine', n_jobs=4)
KNN_model.fit(train_tfidf, y_train)

# Scoring model, printing scores
print(f'Train score: {KNN_model.score(train_tfidf, y_train)}')
print(f'Validation score: {KNN_model.score(val_tfidf, y_val)}')
print(f'Test score: {KNN_model.score(test_tfidf, y_test)}')


Train score: 0.6902575792754572
Validation score: 0.5686605597534721
Test score: 0.5685135349058097


Our `validation` and `test` scores differ from our `train` score by 0.13, which is an indicator that our model is overfit.

The most obvious place to look when faced with an overfit KNN model is the number of neighbors. Higher numbers of neighbors lead to overfitting, so we will only be checking `n_neighbors` values **below** the default, which is 5. Below we will run loop through the different `n_neighbors` values and check the corresponding scores.

In [18]:
neighbors = range(1, 5)  # range between 1,4

train_acc = [] # empty lists
val_acc = []

# looping through neighbours to find best fit
for n in neighbors: 
    print(f"Calculating score with {n} neighbor/s...", end="\r")
    
    #Instantiating the model & fitting to data
    KNN_model = KNeighborsClassifier(n_neighbors=n, metric='cosine', n_jobs=4)
    KNN_model.fit(train_tfidf, y_train)
    
    #Scoring model on train and val
    train_accuracy = KNN_model.score(train_tfidf, y_train)
    val_accuracy = KNN_model.score(val_tfidf, y_val)
    
    #appending lists with accuracies
    train_acc.append(train_accuracy)
    val_acc.append(val_accuracy)

In [19]:
#Trick to find the ideal n value: 

# indexing highest accuracy
index_of_max = np.argmax(val_acc)

# the corresponding best n value
best_n = neighbors[index_of_max]

best_n

3

Our best `n` is 3, so we will now pass that into our KNN model to see if it mitigates our overfitting problem.

In [21]:
# running model with updated nearest neighbors
KNN_model = KNeighborsClassifier(n_neighbors=3, metric='cosine', n_jobs=4)
KNN_model.fit(train_tfidf, y_train)

# Scoring and printing scores
print(f'Train score: {KNN_model.score(train_tfidf, y_train)}')
print(f'Validation score: {KNN_model.score(val_tfidf, y_val)}')


Train score: 0.7545292722910555
Validation score: 0.5875300772510448


A `n_neighbors` of 3 got us a significant boost in our `train` score, as well as a marginal boost to our `validation` score. Our overfitting, however, is now worse. Maybe there's a fault in my logic..